In [1]:
import tensorflow as tf
# import cv2
import tensorflow.keras as K
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,LeakyReLU,BatchNormalization,Activation, MaxPooling2D, Dropout, Dense, Flatten, \
    GlobalAveragePooling2D,  SeparableConv2D, GlobalMaxPooling2D,Input,Layer,add


In [6]:
def cbl_cell(img_input,filters,kernels,stride=(1,1)):
    origin = Conv2D(filters, kernels,strides =stride,use_bias=False,padding='SAME')(img_input)
    x = BatchNormalization()(origin)
    x = LeakyReLU(alpha=0.05)(x)
    return x

In [3]:
img_input = Input(shape=(441,399,1))
x = cbl_cell(img_input)
x

Metal device set to: Apple M1


2021-11-16 22:15:10.336586: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-16 22:15:10.336844: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<KerasTensor: shape=(None, 441, 399, 32) dtype=float32 (created by layer 'leaky_re_lu')>

In [3]:
def res1(inputs,filter_num,stride=(1,1)):
    x = cbl_cell(inputs,filter_num,stride)
    x = cbl_cell(inputs,2*filter_num,stride)
    x = add([inputs,x])
    return x
    

In [4]:
def static_res_block(inputs,filters,n):
    x = res1(inputs, filters)
    for i in range(n - 1):
        x = res1(x, filters)
    return x

In [19]:
x = static_res_block(x,32,2)

In [7]:
img_input = Input(shape=(441,399,1))
x = cbl_cell(img_input,32,(3,3))
x = cbl_cell(x,64,(3,3),(2,2))
x = static_res_block(x,32,1)
x = cbl_cell(x,128,(3,3),(2,2))
x = static_res_block(x,64,2)
x = cbl_cell(x,256,(3,3),(2,2))
x = static_res_block(x,128,8)
x = cbl_cell(x,512,(3,3),(2,2))
x = static_res_block(x,256,8)
x = cbl_cell(x,1024,(3,3),(2,2))
x = static_res_block(x,512,4)
x

<KerasTensor: shape=(None, 14, 13, 1024) dtype=float32 (created by layer 'add_42')>